In [1]:
import tensorflow as tf
import numpy as np
import ntm.ntm_cell as ntm_cell

In [2]:
batch_size = 100
seq_length = 128
vector_dim = 13

rnn_size = 10
memory_size = 128
memory_vector_dim = 13

learning_rate = 0.0001

In [3]:
x = tf.placeholder(name='x', dtype=tf.float32, shape=[batch_size, seq_length, vector_dim])
y = x
#eof = np.zeros([args.batch_size, args.vector_dim + 1])
#eof[:, args.vector_dim] = np.ones([args.batch_size])
#eof = tf.constant(eof, dtype=tf.float32)
zero = tf.constant(np.zeros([batch_size, vector_dim]), dtype=tf.float32)

cell = ntm_cell.NTMCell(rnn_size, memory_size, memory_vector_dim, 1, 1,
                            addressing_mode='content_and_location',
                            reuse=False,
                            output_dim=vector_dim)

state = cell.zero_state(batch_size, tf.float32)
state_list = [state]
for t in range(seq_length):
    #output, state = cell(tf.concat([x[:, t, :], np.zeros([batch_size, 1])], axis=1), state)
    output, state = cell(x[:, t, :], state)
    state_list.append(state)
#output, state = cell(eof, state)
#self.state_list.append(state)

o = []
for t in range(seq_length):
    output, state = cell(zero, state)
    o.append(output[:, 0:vector_dim])
    state_list.append(state)
o = tf.sigmoid(tf.transpose(o, perm=[1, 0, 2]))

# self.copy_loss = tf.reduce_mean(tf.reduce_sum(tf.square(self.y - self.o), reduction_indices=[1, 2]))
eps = 1e-8
copy_loss = -tf.reduce_mean(   # cross entropy function
                y * tf.log(o + eps) + (1 - y) * tf.log(1 - o + eps)
                           )

with tf.variable_scope('optimizer', reuse=False):
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate, momentum=0.9, decay=0.95)
    gvs = optimizer.compute_gradients(copy_loss)
    capped_gvs = [(tf.clip_by_value(grad, -10., 10.), var) for grad, var in gvs]
    train_op = optimizer.apply_gradients(capped_gvs)

copy_loss_summary = tf.summary.scalar('copy_loss_%d' % seq_length, copy_loss)
# self.merged_summary = tf.summary.merge(self.copy_loss_summary)

In [5]:
def generate_random_strings(batch_size, seq_length, vector_dim):
    return np.random.randint(0, 2, (batch_size, seq_length, vector_dim))

In [7]:
sess = tf.Session()

for epo in range(10000):
    X = generate_random_strings(batch_size, seq_length, vector_dim)
    if epo % 100 == 0:        # test
        p = 0               # select p th sample in the batch to show
        print(X[p, :, :])
        
        outp, states, cpl, cpls = sess.run([o, state_list, copy_loss, copy_loss_summary], feed_dict={x: X})[p, :, :]
        
        print(outp)
        
        w_plot = []
        M_plot = np.concatenate([state['M'][p, :, :] for state in states])
        for state in states:
            w_plot.append(np.concatenate([state['w_list'][0][p, :], state['w_list'][1][p, :]]))
        plt.imshow(w_plot, interpolation='nearest', cmap='gray')
        plt.draw()
        plt.pause(0.001)
        
        print('batches %d, loss %g' % (epo, copy_loss))
    else:                   # train
        sess.run(train_op, feed_dict={'x': X})

[[1 1 0 ..., 1 0 0]
 [1 1 0 ..., 1 1 0]
 [0 1 0 ..., 1 0 1]
 ..., 
 [0 0 1 ..., 0 1 0]
 [0 0 1 ..., 1 0 0]
 [1 0 0 ..., 0 1 0]]


MemoryError: 